# 03 — Data Preparation (CRISP-DM)

Nesta etapa do CRISP-DM, os dados brutos são preparados para análise e
modelagem, incluindo limpeza, padronização, tratamento de valores ausentes
e anonimização de informações sensíveis.

O objetivo é gerar uma base consistente, confiável e reutilizável para
as próximas fases do projeto.

In [26]:
import numpy as np
import pandas as pd
from pathlib import Path

In [27]:
DATA_RAW_PATH = Path("../data/raw")

csv_files = sorted(DATA_RAW_PATH.glob("*.csv"))

dfs = []

for file in csv_files:
    df = pd.read_csv(file, encoding="latin1", sep=";")
    df["arquivo_origem"] = file.name
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

df.shape

(24009, 16)

In [28]:
def standardize_columns(df):
    df = df.copy()
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.normalize("NFKD")
        .str.encode("ascii", errors="ignore")
        .str.decode("utf-8")
        .str.replace(r"[^a-z0-9_]", "_", regex=True)
    )

    return df

df.columns

Index(['Referência', 'Nome', 'Cargo', 'Data Admissão', 'Tipo de Regime',
       'Descontos', 'Liquido', 'Data Desligamento', 'Proventos', 'Contrato',
       'Atividade', 'Nome Atividade', 'Tipo de Contrato',
       'Data Prevista Termino Contrato', 'Carga Horária (Sem.)',
       'arquivo_origem'],
      dtype='object')

## Renomeando colunas

In [29]:
# Criando df_prepared como cópia de df_raw
df_prepared = df.copy()

# renomeando colunas
df_prepared.rename(columns={
    "Referência": "referencia",
    "Nome": "nome",
    "Cargo": "cargo",
    "Data Admissão": "data_admissao",
    "Tipo de Regime": "tipo_regime",
    "Descontos": "descontos",
    "Liquido": "liquido",
    "Data Desligamento": "data_desligamento",
    "Proventos": "proventos",
    "Contrato": "contrato",
    "Atividade": "atividade",
    "Nome Atividade": "nome_atividade",
    "Tipo de Contrato": "tipo_contrato",
    "Data Prevista Termino Contrato": "data_prevista_termino_contrato",
    "Carga Horária (Sem.)": "carga_horaria_semanal"
}, inplace=True)

In [30]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,atividade,nome_atividade,tipo_contrato,data_prevista_termino_contrato,carga_horaria_semanal,arquivo_origem
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,40.0,abr2025.csv
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,"1.584,02","3.603,75",NaN,"5.187,77",1.0,NaN,NaN,Efetivo,NaN,40.0,abr2025.csv
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,30.0,abr2025.csv
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,"1.330,84","5.432,23",NaN,"6.763,07",1.0,NaN,NaN,Efetivo,NaN,30.0,abr2025.csv
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,40.0,abr2025.csv


## Removendo as linhas de somas finais que vem em cada dataset

In [31]:
df_prepared = df_prepared.dropna(subset=["referencia"])

In [32]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,atividade,nome_atividade,tipo_contrato,data_prevista_termino_contrato,carga_horaria_semanal,arquivo_origem
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,40.0,abr2025.csv
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,"1.584,02","3.603,75",NaN,"5.187,77",1.0,NaN,NaN,Efetivo,NaN,40.0,abr2025.csv
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,30.0,abr2025.csv
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,"1.330,84","5.432,23",NaN,"6.763,07",1.0,NaN,NaN,Efetivo,NaN,30.0,abr2025.csv
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,40.0,abr2025.csv


In [33]:
# Extrair tipo de pagamento e mês diretamente da coluna referencia
df_prepared[["tipo_pagamento_raw", "mes_referencia"]] = df_prepared["referencia"].str.split(" - ", expand=True)

# Padronizar tipo de pagamento
df_prepared["tipo_pagamento"] = df_prepared["tipo_pagamento_raw"].replace({
    "Folha Mensal": "folha_mensal",
    "Folha Complementar": "vale_alimentacao"
})

# remover coluna temporária
df_prepared = df_prepared.drop(columns=["tipo_pagamento_raw"])

# conferir resultados
df_prepared[["referencia", "tipo_pagamento", "mes_referencia"]].head()

,referencia,tipo_pagamento,mes_referencia
0,Folha Complementar - Abril,vale_alimentacao,Abril
1,Folha Mensal - Abril,folha_mensal,Abril
2,Folha Complementar - Abril,vale_alimentacao,Abril
3,Folha Mensal - Abril,folha_mensal,Abril
4,Folha Complementar - Abril,vale_alimentacao,Abril


In [34]:
df_prepared.tail()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,atividade,nome_atividade,tipo_contrato,data_prevista_termino_contrato,carga_horaria_semanal,arquivo_origem,mes_referencia,tipo_pagamento
24003,Folha Mensal - Setembro,YASMIN MICKAELA TAMINI,AGENTE DE DESENVOLVIMENTO INFANTIL,17/01/2025,REGIME PROPRIO,"535,56","2.920,63",NaN,"3.456,19",1.0,NaN,NaN,Efetivo,NaN,40.0,set2025.csv,Setembro,folha_mensal
24004,Folha Complementar - Setembro,ZAIRA ANGELINA ROGADO,AUXILIAR DE CRECHE,09/12/2013,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,40.0,set2025.csv,Setembro,vale_alimentacao
24005,Folha Mensal - Setembro,ZAIRA ANGELINA ROGADO,AUXILIAR DE CRECHE,09/12/2013,REGIME PROPRIO,"323,08","2.442,78",NaN,"2.765,86",1.0,NaN,NaN,Efetivo,NaN,40.0,set2025.csv,Setembro,folha_mensal
24006,Folha Mensal - Setembro,ZILDA MESSINA,AUXILIAR DE ENFERMAGEM,02/05/1994,REGIME PROPRIO,"1.947,27","3.576,00",NaN,"5.523,27",1.0,NaN,NaN,Efetivo,NaN,30.0,set2025.csv,Setembro,folha_mensal
24007,Folha Complementar - Setembro,ZILDA MESSINA,AUXILIAR DE ENFERMAGEM,02/05/1994,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,30.0,set2025.csv,Setembro,vale_alimentacao


## Inferência manual do sexo dos servidores — Processo

### Extração dos nomes
- A partir do dataset preparado, foram extraídos os nomes dos servidores.
- Os nomes foram deduplicados para garantir que cada servidor fosse analisado apenas uma vez.

### Normalização dos nomes
- Os nomes foram padronizados para facilitar comparações posteriores.
- A normalização consistiu em:
  - Conversão para letras maiúsculas
  - Remoção de espaços em branco no início e no fim do texto

### Criação de DataFrame auxiliar
- Foi criado um DataFrame exclusivo contendo:
  - Nome do servidor
  - Campo vazio para inferência manual do sexo
- Esse DataFrame passou a ser a fonte oficial da variável inferida.

### Exportação para CSV
- O DataFrame auxiliar foi exportado para um arquivo CSV.
- O objetivo foi permitir uma inferência manual mais eficiente em ambiente visual (ex.: Excel ou LibreOffice).

### Inferência manual do sexo
- Cada nome foi analisado individualmente.
- O sexo foi inferido com base no primeiro nome do servidor.
- Foi adotada a seguinte convenção:
  - `M` para masculino
  - `F` para feminino

### Reimportação do arquivo tratado
- Após a inferência manual, o arquivo CSV foi reimportado para o ambiente de análise.
- A normalização dos nomes foi reaplicada para garantir consistência.

### Integração com o dataset principal
- A variável `sexo_inferido` foi incorporada ao dataset principal por meio de *merge*.
- A junção foi realizada utilizando o nome normalizado como chave.

### Validação final
- Foi realizada a verificação de valores nulos na variável `sexo_inferido`.
- Caso identificados, os registros foram corrigidos diretamente no CSV e o processo de importação e *merge* foi repetido.


In [35]:
nomes_unicos = (
    df_prepared["nome"]
    .dropna()
    .drop_duplicates()
    .sort_values()
    .reset_index(drop=True)
)

nomes_unicos

0      ACACIO DONIZETTI DE OLIVEIRA
1                 ADA ESTER ARCHILA
2           ADAO RODRIGUES DE JESUS
3                 ADELINI PELINGRIN
4                ADEMILTON MONTEIRO
                   ...             
975            WILSON RANOEL VERGNA
976               YARA MARIA ISABEL
977          YASMIN MICKAELA TAMINI
978           ZAIRA ANGELINA ROGADO
979                   ZILDA MESSINA
Name: nome, Length: 980, dtype: object

In [36]:
df_sexo_servidores = pd.DataFrame({
    "nome_servidor": nomes_unicos,
    "sexo_inferido": pd.NA
})

df_sexo_servidores

,nome_servidor,sexo_inferido
0,ACACIO DONIZETTI DE OLIVEIRA,<NA>
1,ADA ESTER ARCHILA,<NA>
2,ADAO RODRIGUES DE JESUS,<NA>
3,ADELINI PELINGRIN,<NA>
4,ADEMILTON MONTEIRO,<NA>
...,...,...
975,WILSON RANOEL VERGNA,<NA>
976,YARA MARIA ISABEL,<NA>
977,YASMIN MICKAELA TAMINI,<NA>
978,ZAIRA ANGELINA ROGADO,<NA>


In [57]:
path_output = Path("../data/interim/sexo_servidores.csv")

df_sexo_servidores.to_csv(
    path_output,
    index=False,
    encoding="utf-8"
)

In [63]:
df_sexo_serv = pd.read_csv("../data/interim/inf_sexo_servidor.csv", sep=";")

df_sexo_serv.head()

,nome_servidor,sexo_inferido
0,ACACIO DONIZETTI DE OLIVEIRA,M
1,ADA ESTER ARCHILA,F
2,ADAO RODRIGUES DE JESUS,M
3,ADELINI PELINGRIN,F
4,ADEMILTON MONTEIRO,M


In [64]:
def normalizar_nome(nome):
    return (
        nome
        .str.upper()
        .str.strip()
    )

df_prepared["nome_servidor_norm"] = normalizar_nome(df_prepared["nome"])
df_sexo_serv["nome_servidor_norm"] = normalizar_nome(df_sexo_serv["nome_servidor"])

In [70]:
df_prepared = df_prepared.drop(columns=["sexo_inferido_x", "sexo_inferido_y"])

In [72]:
df_prepared = df_prepared.merge(
    df_sexo_serv[["nome_servidor_norm", "sexo_inferido"]],
    on="nome_servidor_norm",
    how="left"
)

In [73]:
df_prepared["sexo_inferido"].value_counts(dropna=False)

sexo_inferido
F    14784
M     9213
Name: count, dtype: int64

In [75]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,atividade,nome_atividade,tipo_contrato,data_prevista_termino_contrato,carga_horaria_semanal,arquivo_origem,mes_referencia,tipo_pagamento,nome_servidor_norm,sexo_inferido
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,40.0,abr2025.csv,Abril,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,"1.584,02","3.603,75",NaN,"5.187,77",1.0,NaN,NaN,Efetivo,NaN,40.0,abr2025.csv,Abril,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,30.0,abr2025.csv,Abril,vale_alimentacao,ADA ESTER ARCHILA,F
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,"1.330,84","5.432,23",NaN,"6.763,07",1.0,NaN,NaN,Efetivo,NaN,30.0,abr2025.csv,Abril,folha_mensal,ADA ESTER ARCHILA,F
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,40.0,abr2025.csv,Abril,vale_alimentacao,ADAO RODRIGUES DE JESUS,M
